In [1]:
# tests/test_routes_provider_manual.py
# Run:  python -m tests.test_routes_provider_manual

import os, sys, json, time, traceback

# Optional: load .env for local runs
try:
    from dotenv import load_dotenv
    load_dotenv()
except Exception:
    pass

# Make project root importable (assumes this file lives in tests/)
os.chdir("..")
sys.path.insert(0, os.getcwd())
from modules.routes_provider import ORSConfig, ORSClient  # or cabosupernet.routes_provider

# ──────────────────────────────────────────────────────────────────────────────
# Helpers
# ──────────────────────────────────────────────────────────────────────────────
def _strip_keys(obj, keys_to_strip={"segments"}):
    """Recursively remove noisy keys from dicts/lists."""
    if isinstance(obj, dict):
        return {k: _strip_keys(v, keys_to_strip)
                for k, v in obj.items() if k not in keys_to_strip}
    if isinstance(obj, list):
        return [_strip_keys(v, keys_to_strip) for v in obj]
    return obj

def jprint(obj) -> None:
    """Pretty-print dicts/lists like JSON, with heavy keys removed."""
    clean = _strip_keys(obj, {"segments"})  # <- omit segments
    print(json.dumps(clean, ensure_ascii=False, indent=2, sort_keys=True))

def run_case(label: str, origin, destination, **kwargs):
    """Run one routing case with timing and robust error display."""
    print("\n" + "=" * 80)
    print(f"{label}")
    print("-" * 80)
    print("origin:", origin)
    print("destination:", destination)
    t0 = time.time()
    try:
        resp = ors.route_road(origin, destination, **kwargs)
        elapsed = (time.time() - t0) * 1000
        print(f"\n✓ OK in {elapsed:.0f} ms — result:\n")
        jprint(resp)
    except Exception as e:
        elapsed = (time.time() - t0) * 1000
        print(f"\n✗ ERROR in {elapsed:.0f} ms — details:\n")
        body = None
        resp_obj = getattr(e, "response", None)
        if resp_obj is not None:
            try:
                body = resp_obj.json()
            except Exception:
                body = resp_obj.text
        err_info = {
            "error_type": type(e).__name__,
            "http_status": getattr(resp_obj, "status_code", None),
            "server_body": body,
            "exception_str": str(e),
        }
        jprint(err_info)
        print("\nTraceback (for debugging):")
        traceback.print_exc()

# ──────────────────────────────────────────────────────────────────────────────
# Guard: require ORS_API_KEY
# ──────────────────────────────────────────────────────────────────────────────
if not os.getenv("ORS_API_KEY"):
    raise SystemExit("Set ORS_API_KEY in your environment or .env before running this test.")

# Optional: bump logging from the client (INFO/DEBUG/WARNING/ERROR)
os.environ.setdefault("ORS_LOG_LEVEL", "INFO")

# ──────────────────────────────────────────────────────────────────────────────
# Client config (defaults to driving-hgv for your TF)
# ──────────────────────────────────────────────────────────────────────────────
cfg = ORSConfig(default_country="BR")
print("=== ORS Client Configuration ===")
print("Profile:", cfg.default_profile)
try:
    # These exist when using driving-hgv defaults from the module rewrite
    print("Truck vehicle_type:", getattr(cfg, "hgv_vehicle_type", None))
    print("Truck restrictions:", json.dumps(getattr(cfg, "hgv_restrictions", None), ensure_ascii=False, sort_keys=True))
    print("Avoid features (default):", getattr(cfg, "avoid_features_default", None))
except Exception:
    pass

ors = ORSClient(cfg)

# ──────────────────────────────────────────────────────────────────────────────
# Test cases
# ──────────────────────────────────────────────────────────────────────────────

# 1) Free-text address
run_case(
    "1) Free-text address",
    "Av. Paulista, 1578, São Paulo, SP",
    "Porto de Santos, Santos, SP",
)

# 2) CEP-only (postal code)
run_case(
    "2) CEP-only (postal code)",
    "01310-200",
    "11010-091",
)

# 3) City names
run_case(
    "3) City names",
    "São Paulo, SP",
    "Rio de Janeiro, RJ",
)

# 4) Coordinates (tuple) → CEP
run_case(
    "4) Coordinates (tuple) → CEP",
    (-23.55052, -46.63331),
    "11010913",
)

# 5) Structured dict → explicit lat/lon
origin_struct = {
    "street": "Avenida Paulista",
    "housenumber": "1578",
    "locality": "São Paulo",
    "region": "SP",
    "postalcode": "01310-200",
    "country": "BR",
}
run_case(
    "5) Structured dict",
    origin_struct,
    {"lat": -23.9608, "lon": -46.3336, "label": "Santos Ponta da Praia"},
)


[10:25:26] INFO cabosupernet.routes_provider | GEOCODE text='Av. Paulista, 1578, São Paulo, SP' country=BR size=1
[10:25:26] INFO cabosupernet.routes_provider | RESOLVE (text) -> {'lat': -23.555036, 'lon': -46.663479, 'label': 'Avenida Paulista, São Paulo, Brazil'}
[10:25:26] INFO cabosupernet.routes_provider | GEOCODE text='Porto de Santos, Santos, SP' country=BR size=1
[10:25:26] INFO cabosupernet.routes_provider | RESOLVE (text) -> {'lat': -23.932511, 'lon': -46.324739, 'label': 'Alfandega do Porto de Santos, Santos, SP, Brazil'}
[10:25:26] INFO cabosupernet.routes_provider | ROUTE try1 driving-hgv coords=[[-46.663479, -23.555036], [-46.324739, -23.932511]]


=== ORS Client Configuration ===
Profile: driving-hgv
Truck vehicle_type: goods
Truck restrictions: {"axleload": 10000, "hazmat": false, "height": 4.2, "length": 18.6, "weight": 40000, "width": 2.6}
Avoid features (default): ['ferries']

1) Free-text address
--------------------------------------------------------------------------------
origin: Av. Paulista, 1578, São Paulo, SP
destination: Porto de Santos, Santos, SP


[10:25:29] INFO cabosupernet.routes_provider | ROUTE OK km=83.28 hrs=1.6
[10:25:29] INFO cabosupernet.routes_provider | GEOCODE structured params={"country": "BR", "postalcode": "01310200", "size": 1}
[10:25:29] INFO cabosupernet.routes_provider | RESOLVE (CEP ViaCEP->text) query='Avenida Paulista, Bela Vista, São Paulo, SP'
[10:25:29] INFO cabosupernet.routes_provider | GEOCODE text='Avenida Paulista, Bela Vista, São Paulo, SP' country=BR size=1
[10:25:29] INFO cabosupernet.routes_provider | RESOLVE (CEP via ViaCEP) -> {'lat': -23.563498, 'lon': -46.653985, 'label': 'Ciclovia da Avenida Paulista, São Paulo, Brazil'}
[10:25:29] INFO cabosupernet.routes_provider | GEOCODE structured params={"country": "BR", "postalcode": "11010091", "size": 1}
[10:25:29] INFO cabosupernet.routes_provider | RESOLVE (CEP ViaCEP->text) query='Rua Frei Gaspar, Centro, Santos, SP'
[10:25:29] INFO cabosupernet.routes_provider | GEOCODE text='Rua Frei Gaspar, Centro, Santos, SP' country=BR size=1
[10:25:29] IN


✓ OK in 2680 ms — result:

{
  "destination": {
    "label": "Alfandega do Porto de Santos, Santos, SP, Brazil",
    "lat": -23.932511,
    "lon": -46.324739
  },
  "distance_m": 83282.4,
  "duration_s": 5742.6,
  "origin": {
    "label": "Avenida Paulista, São Paulo, Brazil",
    "lat": -23.555036,
    "lon": -46.663479
  }
}

2) CEP-only (postal code)
--------------------------------------------------------------------------------
origin: 01310-200
destination: 11010-091


[10:25:30] INFO cabosupernet.routes_provider | ROUTE OK km=83.74 hrs=1.58
[10:25:30] INFO cabosupernet.routes_provider | GEOCODE text='São Paulo, SP' country=BR size=1
[10:25:30] INFO cabosupernet.routes_provider | RESOLVE (text) -> {'lat': -23.570533, 'lon': -46.663713, 'label': 'São Paulo, Brazil'}
[10:25:30] INFO cabosupernet.routes_provider | GEOCODE text='Rio de Janeiro, RJ' country=BR size=1
[10:25:30] INFO cabosupernet.routes_provider | RESOLVE (text) -> {'lat': -22.935024, 'lon': -43.518246, 'label': 'Rio de Janeiro, Brazil'}
[10:25:30] INFO cabosupernet.routes_provider | ROUTE try1 driving-hgv coords=[[-46.663713, -23.570533], [-43.518246, -22.935024]]


...


✓ OK in 6336 ms — result:

{
  "destination": {
    "label": "Rio de Janeiro, Brazil",
    "lat": -22.935024,
    "lon": -43.518246
  },
  "distance_m": 405399.4,
  "duration_s": 23464.4,
  "origin": {
    "label": "São Paulo, Brazil",
    "lat": -23.570533,
    "lon": -46.663713
  }
}

4) Coordinates (tuple) → CEP
--------------------------------------------------------------------------------
origin: (-23.55052, -46.63331)
destination: 11010913


[10:25:37] INFO cabosupernet.routes_provider | ROUTE OK km=70.0 hrs=1.4
[10:25:37] INFO cabosupernet.routes_provider | GEOCODE structured params={"address": "1578 Avenida Paulista", "country": "BR", "locality": "São Paulo", "postalcode": "01310-200", "region": "SP", "size": 1}
[10:25:37] INFO cabosupernet.routes_provider | RESOLVE (structured dict) -> {'lat': -23.555036, 'lon': -46.663479, 'label': 'Avenida Paulista, São Paulo, Brazil'}
[10:25:37] INFO cabosupernet.routes_provider | RESOLVE (dict lat/lon) -> {'lat': -23.9608, 'lon': -46.3336, 'label': 'Santos Ponta da Praia'}
[10:25:37] INFO cabosupernet.routes_provider | ROUTE try1 driving-hgv coords=[[-46.663479, -23.555036], [-46.3336, -23.9608]]



✓ OK in 1015 ms — result:

{
  "destination": {
    "label": "Centro, Santos, SP, Brazil",
    "lat": -23.935639,
    "lon": -46.325439
  },
  "distance_m": 70002.9,
  "duration_s": 5048.2,
  "origin": {
    "label": "-23.550520,-46.633310",
    "lat": -23.55052,
    "lon": -46.63331
  }
}

5) Structured dict
--------------------------------------------------------------------------------
origin: {'street': 'Avenida Paulista', 'housenumber': '1578', 'locality': 'São Paulo', 'region': 'SP', 'postalcode': '01310-200', 'country': 'BR'}
destination: {'lat': -23.9608, 'lon': -46.3336, 'label': 'Santos Ponta da Praia'}


[10:25:39] INFO cabosupernet.routes_provider | ROUTE OK km=86.71 hrs=1.68



✓ OK in 1441 ms — result:

{
  "destination": {
    "label": "Santos Ponta da Praia",
    "lat": -23.9608,
    "lon": -46.3336
  },
  "distance_m": 86706.2,
  "duration_s": 6041.5,
  "origin": {
    "label": "Avenida Paulista, São Paulo, Brazil",
    "lat": -23.555036,
    "lon": -46.663479
  }
}
